Implementation of Deep net with 2 hidden layers that learns with encrypted data

In [1]:
!pip install tenseal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/4.8 MB ? eta -:--:--

     ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.6/4.8 MB 17.1 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 3.0/4.8 MB 28.9 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 4.8/4.8 MB 35.7 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 28.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
import tenseal as ts
import pandas as pd
import random
from time import time

# those are optional and are not necessary for training
import numpy as np
import matplotlib.pyplot as plt

In [3]:
torch.random.manual_seed(73)
random.seed(73)


def split_train_test(x, y, test_ratio=0.3):
    idxs = [i for i in range(len(x))]
    random.shuffle(idxs)
    # delimiter between test and train data
    delim = int(len(x) * test_ratio)
    test_idxs, train_idxs = idxs[:delim], idxs[delim:]
    return x[train_idxs], y[train_idxs], x[test_idxs], y[test_idxs]


def heart_disease_data():
    data = pd.read_csv("/kaggle/input/trydataset/framingham.csv")
    # drop rows with missing values
    data = data.dropna()
    # drop some features
    data = data.drop(columns=["education", "currentSmoker", "BPMeds", "diabetes", "diaBP", "BMI"])
    # balance data
    grouped = data.groupby('TenYearCHD')
    data = grouped.apply(lambda x: x.sample(grouped.size().min(), random_state=73).reset_index(drop=True))
    # extract labels
    y = torch.tensor(data["TenYearCHD"].values).float().unsqueeze(1)
    #data = data.drop("TenYearCHD",'columns')
    # standardize data
    data = (data - data.mean()) / data.std()
    x = torch.tensor(data.values).float()
    return split_train_test(x, y)


def random_data(m=1024, n=2):
    # data separable by the line `y = x`
    x_train = torch.randn(m, n)
    x_test = torch.randn(m // 2, n)
    y_train = (x_train[:, 0] >= x_train[:, 1]).float().unsqueeze(0).t()
    y_test = (x_test[:, 0] >= x_test[:, 1]).float().unsqueeze(0).t()
    return x_train, y_train, x_test, y_test


# You can use whatever data you want without modification to the tutorial
# x_train, y_train, x_test, y_test = random_data()
x_train, y_train, x_test, y_test = heart_disease_data()

print("############# Data summary #############")
print(f"x_train has shape: {x_train.shape}")
print(f"y_train has shape: {y_train.shape}")
print(f"x_test has shape: {x_test.shape}")
print(f"y_test has shape: {y_test.shape}")
print("#######################################")

/tmp/ipykernel_74/2663072693.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = grouped.apply(lambda x: x.sample(grouped.size().min(), random_state=73).reset_index(drop=True))


############# Data summary #############
x_train has shape: torch.Size([780, 10])
y_train has shape: torch.Size([780, 1])
x_test has shape: torch.Size([334, 10])
y_test has shape: torch.Size([334, 1])
#######################################


In [4]:
# parameters
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
# create TenSEALContext
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_training.global_scale = 2 ** 21
ctx_training.generate_galois_keys()

In [5]:
#t_start = time()
#enc_x_train = ts.ckks_tensor(ctx_training, x_train[0:200])
#enc_y_train = ts.ckks_tensor(ctx_training, y_train[0:200])
#t_end = time()
#print(f"Encryption of the training_set took {int(t_end - t_start)} seconds")
#print(enc_x_train[0])

In [6]:
class EncryptedDL2layer() :
  def __init__(self) -> None:
      self.weight1=np.random.rand(10,6)*0.01
      self.bias1=np.random.rand(6)*0.01
      self.weight2=np.random.rand(6, 3)*0.01
      self.bias2=np.random.rand(3)*0.01
      self.weight3=np.random.rand(3,1)*0.01
      self.bias3=np.random.rand(1)*0.01
      self.dw1=0
      self.db1=0
      self.dw2=0
      self.db2=0
      self.dw3=0
      self.db3=0
  def bootstrapping(enc,ctx_training) :
    return ts.ckks_tensor(ctx_training,enc.decrypt())
  def sigmoid(enc_x):
    return enc_x.polyval([0.5, 0.197, 0, -0.004])
  def sigmoid_derv(enc_x):
    return enc_x.polyval([0.197,0,-0.012,0])
  def forward(self,enc_x_train,ctx_training) :
    z11=enc_x_train.mm(self.weight1)
    z1=z11.add(self.bias1)
    a1=EncryptedDL2layer.sigmoid(z1)
    y=EncryptedDL2layer.bootstrapping(a1,ctx_training)
    z21=y.mm(self.weight2)
    z2=z21.add(self.bias2)
    a2=EncryptedDL2layer.sigmoid(z2)
    y1=EncryptedDL2layer.bootstrapping(a2,ctx_training)
    z31=y1.mm(self.weight3)
    z3=z31.add(self.bias3)
    a3=EncryptedDL2layer.sigmoid(z3)
    a3=EncryptedDL2layer.bootstrapping(a3,ctx_training)
    return a3,z3,a2,z2,a1,z1
  def backward(self,a3,z3,a2,z2,a1,z1,enc_y_train,ctx_training) :
    #calculating the output at the layer 2
    error=a3-enc_y_train
    der=EncryptedDL2layer.sigmoid_derv(z3)
    #finding delta3
    delta3=error.mul(der)
    #using bootstrapping
    delta3=EncryptedDL2layer.bootstrapping(delta3,ctx_training)

    del2=delta3.mm(self.weight3.transpose())
    del2=EncryptedDL2layer.bootstrapping(del2,ctx_training)
    #finding der2
    der2=EncryptedDL2layer.sigmoid_derv(z2)
    #using bootstrapping
    der2=EncryptedDL2layer.bootstrapping(der2,ctx_training)
    #finding delta2
    delta2=del2.mul(der2)
    del1=delta2.mm(self.weight2.transpose())
    del1=EncryptedDL2layer.bootstrapping(del1,ctx_training)
    #finding der1
    der1=EncryptedDL2layer.sigmoid_derv(z1)
    #using bootstrapping
    der1=EncryptedDL2layer.bootstrapping(der1,ctx_training)
    #finding delta1
    delta1=del1.mul(der1)
    #finding the gradients
    #for weight3 and bias3
    self.dw3=a2.transpose().mm(delta3)
    self.db3=delta3.sum()
    #for weight2 and bias2
    self.dw2=a1.transpose().mm(delta2)
    self.db2=delta2.sum()
    #for weight1 and bias1
    self.dw1=enc_x_train.transpose().mm(delta1)
    self.db1=delta1.sum()
  def update_params(self):
    self.weight3=self.weight3-0.1*self.dw3
    self.bias3=self.bias3-0.1*self.db3
    self.weight2=self.weight2-0.1*self.dw2
    self.bias2=self.bias2-0.1*self.db2
    self.weight1=self.weight1-0.1*self.dw1
    self.bias1=self.bias1-0.1*self.db1
  def encrypt(self, context):
    self.weight1 = ts.ckks_tensor(context, self.weight1)
    self.bias1 = ts.ckks_tensor(context, self.bias1)
    self.weight2 = ts.ckks_tensor(context, self.weight2)
    self.bias2 = ts.ckks_tensor(context, self.bias2)
    self.weight3 = ts.ckks_tensor(context, self.weight3)
    self.bias3 = ts.ckks_tensor(context, self.bias3)
  def decrypt(self):
    self.weight1 = self.weight1.decrypt()
    self.bias1 = self.bias1.decrypt()
    self.weight2 = self.weight2.decrypt()
    self.bias2 = self.bias2.decrypt()
    self.weight3 = self.weight3.decrypt()
    self.bias3 = self.bias3.decrypt()
  def accuracy(self,x_test,y_test):
    #self.decrypt()
    w1 = torch.tensor(self.weight1)
    b1 = torch.tensor(self.bias1)
    out1 = torch.sigmoid(x_test.matmul(w1) + b1).reshape(-1, 1)
    w2 = torch.tensor(self.weight2)
    b2 = torch.tensor(self.bias2)
    out2 = torch.sigmoid(out1.matmul(w2) + b2).reshape(-1, 1)
    correct = torch.abs(y_test - out2) < 0.5
    return correct.float().mean()

  def __call__(self, *args, **kwargs):
    return self.forward(*args, **kwargs)


In [7]:
EPOCHS = 10
eelr = EncryptedDL2layer()
times = []
for epoch in range(EPOCHS):
    print("Epoch : ",epoch)
    for i in range(0,780,60):
        t_start = time()
        enc_x_train = ts.ckks_tensor(ctx_training, x_train[i:i+60])
        enc_y_train = ts.ckks_tensor(ctx_training, y_train[i:i+60])
        t_end = time()
        print(f"Encryption of the training_set took {int(t_end - t_start)} seconds")
        print("Batch : ",i/60)
        eelr.encrypt(ctx_training)
        t_start = time()
        a3,z3,a2,z2,a1,z1 = eelr.forward(enc_x_train,ctx_training)
        eelr.backward(a3,z3,a2,z2,a1,z1,enc_y_train,ctx_training)
        eelr.update_params()
        t_end = time()
        times.append(t_end - t_start)
    
        eelr.decrypt()
        print("\n")
    a3,z3,a2,z2,a1,z1 = eelr.forward(enc_x_train,ctx_training)
    data=torch.tensor(a3.decrypt().tolist())
    loss_fn = torch.nn.BCEWithLogitsLoss()
    loss = loss_fn(data, y_train[i:i+60])
    print("Loss at epoch ",epoch,loss.data)
    out=torch.tensor(enc_y_train.sub(a3).decrypt().tolist())
    correct=torch.abs(out)<0.5
    print("Accuracy at epoch ",epoch,correct.float().mean())
print(f"\nAverage time per epoch: {int(sum(times) / len(times))} seconds")
print("Final weight1 ",eelr.weight1.tolist())
print("Final bias 1",eelr.bias1.tolist())
print("Final weight 2",eelr.weight2.tolist())
print("Final bias 2",eelr.bias2.tolist())

Epoch :  0
Encryption of the training_set took 0 seconds
Batch :  0.0


Encryption of the training_set took 0 seconds
Batch :  1.0


Encryption of the training_set took 0 seconds
Batch :  2.0


Encryption of the training_set took 0 seconds
Batch :  3.0


Encryption of the training_set took 0 seconds
Batch :  4.0


...

Encryption of the training_set took 0 seconds
Batch :  12.0


Loss at epoch  9 

tensor(0.5425)


Accuracy at epoch  9 

tensor(1.)

Average time per epoch: 33 seconds
Final weight1  [[0.09635900105348369, 0.08106810661398219, 0.08025965340451009, 0.09324417348771562, 0.05048871403367734, 0.04423454177342805], [0.1287784066196389, 0.08206535361462645, 0.10635632361518846, 0.1372515436442451, 0.09961748951736629, 0.11240632960030233], [0.07558412173118376, 0.07677009657295675, 0.06129029136567401, 0.10050685570837184, 0.085787470275764, 0.0696272671620557], [0.07667456288276853, 0.05542328256672671, 0.04369799560924848, 0.09407538354834112, 0.07478844358084184, 0.0721095389111601], [0.06478119642372437, 0.05478460265956432, 0.029692908599469322, 0.09285922210572856, 0.03208212898103649, 0.08007705535848719], [0.06122064070226025, 0.041900373267889236, 0.024283403261679704, 0.07042682094182483, 0.017229071186967156, 0.044040423057032645], [0.11027900240657638, 0.1282894610514163, 0.09720771074424248, 0.11410700279849192, 0.09644373600332641, 0.08719031002918438], [-0.014349540157513089, 0.0166288229929329,

In [8]:
#eelr.encrypt(ctx_training)
m,n,j,k,l,r=eelr.forward(enc_x_train,ctx_training)
enc_y_train.sub(m).decrypt().tolist()

[[0.1732936429005739],
 [-0.16994255437403527],
 [0.15396130421728196],
 [0.1782582769529296],
 [-0.15838260693014808],
 [0.15908689147117847],
 [0.16774926456611064],
 [-0.011456603055851606],
 [0.15304274529243478],
 [0.19298878608463066],
 [0.13831062842681097],
 [-0.19579654654007717],
 [-0.09059897567860427],
 [-0.055760656947737526],
 [-0.07892495361168388],
 [-0.019005290963151133],
 [-0.11834653838941869],
 [0.16231978877766187],
 [-0.08206282556761994],
 [0.16412941082567942],
 [-0.15725576491485463],
 [-0.11731023924101316],
 [0.1849137519169841],
 [-0.0662370371804126],
 [-0.1692493661522894],
 [-0.07425472493041885],
 [0.0038509710331007655],
 [-0.08284964717619178],
 [0.17817172405141737],
 [0.16027625510642068],
 [0.14468629026470528],
 [-0.07673657326232644],
 [0.17495252851945572],
 [-0.07801085736512098],
 [-0.02725698988714836],
 [-0.13863915373586258],
 [0.13922284584745148],
 [0.15163002985716076],
 [0.18699453523211113],
 [0.165090707414751],
 [0.1578056767478586],

In [9]:
out=torch.tensor(enc_y_train.sub(m).decrypt().tolist())
correct=torch.abs(out)<0.5
print("Train Accuracy : ",correct.float().mean())

Train Accuracy :  

tensor(1.)


In [10]:
enc_x_test = ts.ckks_tensor(ctx_training, x_test)
enc_y_test = ts.ckks_tensor(ctx_training, y_test)

In [11]:
m,n,j,k,l,r=eelr.forward(enc_x_test,ctx_training)
print(m.decrypt().tolist())
out=torch.tensor(enc_y_test.sub(m).decrypt().tolist())
correct=torch.abs(out)<0.5
print("Test Accuracy : ",correct.float().mean())

[[0.808731813564115], [0.7949938112555635], [0.047727019455301674], [0.8522438591689995], [0.09685565215561458], [0.8268828024422988], [0.1944659613103399], [0.007406371326718708], [0.045157328335107355], [0.022164636823519934], [0.8425800000858189], [0.06070938857621762], [0.13542695353270806], [0.8300188000783636], [0.8336799856008787], [0.8530786449621476], [0.8333006459642331], [0.8523806948278358], [0.8285968802438546], [0.04880179272889263], [0.8327144458919323], [0.04660690359487726], [0.12710610087965757], [0.12880641412282098], [0.14904444709383183], [0.02084838968000314], [0.8129034586832915], [0.2447737413116347], [0.8416506089390393], [0.8164962542674318], [0.8376104173406049], [0.026023129042736976], [0.8124875390510581], [0.08308225795299563], [0.11079430881809699], [0.062102451813180734], [0.08307237620147735], [0.050364262931103396], [0.811889149614654], [0.06828147203994345], [0.8320198332095303], [0.8547235927264776], [0.8369654670779184], [0.055936383808721], [0.0759

Test Accuracy :  

tensor(1.)
